In [ ]:
Trans_Dim_Staff

SELECT
    -- Tất cả các cột từ bảng staff
    s.staff_id,
    s.first_name,
    s.last_name,
    s.address_id,
    s.email,
    s.store_id,
    s.active,
    s.picture,
    -- Tất cả các cột từ bảng store
    st.store_id AS store_id,
    st.manager_staff_id,
    st.address_id,

    -- Tất cả các cột từ bảng address
    a.address_id,
    a.address as store_address,
    a.district as store_district,
    a.city_id,
    a.postal_code as store_postal_code,
    a.phone store_phone,

    -- Tất cả các cột từ bảng city
    ci.city_id AS city_id,
    ci.city as store_city,
    ci.country_id,

    -- Tất cả các cột từ bảng country
    co.country_id AS country_id,
    co.country as store_country

FROM
    s  -- Bảng staff chứa thông tin nhân viên
JOIN
    st  -- Bảng store chứa thông tin cửa hàng
    ON s.store_id = st.store_id
JOIN
    a  -- Bảng address chứa thông tin địa chỉ
    ON st.address_id = a.address_id
JOIN
    ci  -- Bảng city chứa thông tin thành phố
    ON a.city_id = ci.city_id
JOIN
    co  -- Bảng country chứa thông tin quốc gia
    ON ci.country_id = co.country_id;


In [ ]:
Trans_DimCustomer

SELECT
    -- Các cột từ bảng customer
    c.customer_id,
    c.store_id,
    c.first_name,
    c.last_name,
    c.email,
    c.address_id,
    c.activebool,
    c.create_date,
    c.active,
    -- Các cột từ bảng address
    a.address_id,
    a.address,
    a.district,
    a.city_id,
    a.postal_code,
    a.phone,
    -- Các cột từ bảng city
    ci.city_id,
    ci.city,
    ci.country_id,
    -- Các cột từ bảng country
    co.country_id,
    co.country

FROM
    c
JOIN
    a ON c.address_id = a.address_id
JOIN
    ci ON a.city_id = ci.city_id
JOIN
    co ON ci.country_id = co.country_id;


In [ ]:
Trans_Film
SELECT
    f.film_id,
    f.title,
    f.description,
    f.release_year,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.replacement_cost,
    f.rating,
    f.language_id,
    l.language_id,
    l.language,
    c.category_id,
    c.category,
    a.actor_id,
    a.actor_first_name,
    a.actor_last_name,
    fc.film_id ,
    fc.category_id,
    fa.film_id,
    fa.actor_id

FROM
    f  -- alias cho bảng film
JOIN
    l  -- alias cho bảng language
    ON f.language_id = l.language_id
JOIN
    fc  -- alias cho bảng film_category
    ON f.film_id = fc.film_id
JOIN
    c  -- alias cho bảng category
    ON fc.category_id = c.category_id
JOIN
    fa  -- alias cho bảng film_actor
    ON f.film_id = fa.film_id
JOIN
    a  -- alias cho bảng actor
    ON fa.actor_id = a.actor_id;


In [ ]:
Payment
SELECT
    payment_id,
    customer_id,
    staff_id,
    rental_id,
    amount,
    CAST(payment_date AS DATE) AS date,  -- Tách phần ngày
    SUBSTRING(CAST(payment_date AS STRING), 12, 8 ) AS time  -- Tách phần giờ từ chuỗi
FROM
    payment;


In [ ]:
Payment
WITH DateWithID AS (
    SELECT
        ROW_NUMBER() OVER(ORDER BY date, time) AS date_id,  -- Tạo cột date_id
        date,  -- Sử dụng cột date có sẵn
        time,  -- Sử dụng cột time có sẵn
        EXTRACT(DAY FROM date) AS day,  -- Ngày trong tháng

        -- Sử dụng WEEKDAY() để tính ngày trong tuần
        WEEKDAY(date) AS day_of_week,  -- Thứ Hai = 0, Chủ Nhật = 6

        EXTRACT(MONTH FROM date) AS month,  -- Tháng
        EXTRACT(YEAR FROM date) AS year,  -- Năm

        -- Xác định cuối tuần (Thứ Bảy và Chủ Nhật)
        CASE
            WHEN WEEKDAY(date) IN (5, 6)
            THEN 'True'
            ELSE 'False'
        END AS is_weekend  -- Xác định cuối tuần
    FROM
        payment
)
SELECT * FROM DateWithID;


In [ ]:
TransFactSales
SELECT
    p.payment_id,       -- ID của giao dịch thanh toán
    p.customer_id,      -- ID của khách hàng thực hiện giao dịch
    p.staff_id,         -- ID của nhân viên phụ trách giao dịch
    p.rental_id,        -- ID của giao dịch thuê phim liên quan
    p.amount,           -- Số tiền thanh toán
    p.payment_date,     -- Ngày và giờ của giao dịch thanh toán


    r.rental_date,      -- Ngày và giờ của giao dịch thuê
    r.inventory_id,     -- ID của mặt hàng tồn kho (liên quan đến phim được thuê)
    r.return_date,      -- Ngày và giờ trả lại phim
    i.film_id,          -- ID của phim liên quan đến mặt hàng tồn kho
    i.store_id,         -- ID của cửa hàng mà mặt hàng tồn kho thuộc về

          -- Thời gian cuối cùng dữ liệu tồn kho được cập nhật
    d.date_id,          -- ID của ngày
    d.date,             -- Ngày cụ thể
    d.day,              -- Ngày trong tháng
    d.day_of_week,      -- Ngày trong tuần
    d.is_weekend,       -- Xác định có phải là cuối tuần không (True hoặc False)
    d.month,            -- Tháng của ngày
    d.year              -- Năm của ngày
FROM
    p
JOIN
    r ON p.rental_id = r.rental_id              -- Kết hợp payment với rental qua rental_id
JOIN
    i ON r.inventory_id = i.inventory_id     -- Kết hợp rental với inventory qua inventory_id
JOIN
    d ON p.payment_date = d.date               -- Kết hợp payment với dimDate qua payment_date
